In [2]:
!pip install -q pyomo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 41.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 5.6 MB/s eta 0:00:00


In [3]:
!apt-get install -y -qq glpk-utils

Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 121753 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_5.0-1_amd64.deb ...
Unpacking libglpk40:amd64 (5.0-1) ...
Selecting previously unselected package glpk-utils.
Preparing to unpack .../glpk-utils_5.0-1_amd64.deb ...
Unpacking glpk-utils (5.0-1) ...
Setting up libsuitesparseconfig5:amd64 (1:5.10.1+dfsg-4b

In [4]:
# Importemos pyomo

In [5]:
import pyomo.environ as pyo

In [6]:
# Creamos un diccionario con las demandas de las tiendas

In [7]:
Demanda = {
   'GDA':    943,        # Guadalaja
   'ZAC':   1320,        # Zacatecas
   'AGU':   1694,        # Aguascalientes
   'PVA':   1885,        # Puerto Vallarta
   'TOR':   1700,        # Torreón
   'LEO':   1508         # León
}

In [8]:
Demanda

{'GDA': 943, 'ZAC': 1320, 'AGU': 1694, 'PVA': 1885, 'TOR': 1700, 'LEO': 1508}

In [9]:
# Creamos un diccionario con las ofertas de las plantas de producción

In [10]:
Oferta = {
   'CDMX':   5050,        # Ciudad de México
   'MTY' :   4000         # Monterrey
}

In [11]:
Oferta

{'CDMX': 5050, 'MTY': 4000}

In [12]:
# Creamos un diccionario con las costos de transporte por tonelada
# para desde cada una de las planta hacia cada una de las tiendas

In [13]:
Costos = {
    ('GDA','CDMX'): 1102,
    ('GDA','MTY') : 1614,
    ('ZAC','CDMX'): 1256,
    ('ZAC','MTY') :  930,
    ('AGU','CDMX'):  990,
    ('AGU','MTY') : 1146,
    ('PVA','CDMX'): 1762,
    ('PVA','MTY') : 2276,
    ('TOR','CDMX'): 2024,
    ('TOR','MTY') :  668,
    ('LEO','CDMX'):  758,
    ('LEO','MTY') : 1392
}

In [14]:
Costos

{('GDA', 'CDMX'): 1102,
 ('GDA', 'MTY'): 1614,
 ('ZAC', 'CDMX'): 1256,
 ('ZAC', 'MTY'): 930,
 ('AGU', 'CDMX'): 990,
 ('AGU', 'MTY'): 1146,
 ('PVA', 'CDMX'): 1762,
 ('PVA', 'MTY'): 2276,
 ('TOR', 'CDMX'): 2024,
 ('TOR', 'MTY'): 668,
 ('LEO', 'CDMX'): 758,
 ('LEO', 'MTY'): 1392}

In [15]:
# Creamos una instancia del modelo

In [16]:
modelo = pyo.ConcreteModel()
modelo.dual = pyo.Suffix(direction = pyo.Suffix.IMPORT)

In [17]:
# Paso 1: Definamos las variables
# Toneladas enviadas desde las fabricas hacia las sucursales

In [18]:
FABRICAS = list(Oferta.keys())
SUCURSALES = list(Demanda.keys())

In [19]:
SUCURSALES, FABRICAS

(['GDA', 'ZAC', 'AGU', 'PVA', 'TOR', 'LEO'], ['CDMX', 'MTY'])

# variables de decision

In [20]:
modelo.x = pyo.Var(SUCURSALES, FABRICAS, domain = pyo.NonNegativeReals)

In [21]:
# Paso 2: Definamos la Función Objetivo
# Costo de Transporte: Sumatoria de los productos de los costos de transporte
# por tonelada multiplicada por las toneladas enviadas

In [22]:
modelo.CostoTransporte = pyo.Objective(
    expr = sum([Costos[s,f]*modelo.x[s,f] for s in SUCURSALES for f in FABRICAS]),
    sense = pyo.minimize)

In [23]:
# Paso 3: Definamos las restricciones
# La sumatoria de los envíos de cada fábrica no podía superar
# la oferta de la respectiva fábrica

In [24]:
modelo.restriccionfabrica = pyo.ConstraintList()
for f in FABRICAS:
    modelo.restriccionfabrica.add(sum([modelo.x[s,f] for s in SUCURSALES]) <= Oferta[f])

In [25]:
# La segunda restriccion:
# La sumatoria de los envíos a cada punto de venta,
# debía satisfacer su respectiva demanda particular

In [26]:
modelo.restriccionsucursal = pyo.ConstraintList()
for s in SUCURSALES:
    modelo.restriccionsucursal.add(sum([modelo.x[s,f] for f in FABRICAS]) == Demanda[s])

In [27]:
# Resolvamos el problema de transporte

In [28]:
solucion = pyo.SolverFactory('glpk').solve(modelo)

In [29]:
solucion.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 9696760.0
  Upper bound: 9696760.0
  Number of objectives: 1
  Number of constraints: 8
  Number of variables: 12
  Number of nonzeros: 24
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.004445791244506836
# ----------------------------------------------------------
#   Solution Information
# --------------------------

In [30]:
print("Costo Total de Transporte MX$:", modelo.CostoTransporte())
print("Con los siguientes envíos:")
for f in FABRICAS:
    for s in SUCURSALES:
        if modelo.x[s,f]() > 0:
            print("Envíe desde la fábrica en", f, "a la sucursal en", s, ":", modelo.x[s,f](), "toneladas")

Costo Total de Transporte MX$: 9696760.0
Con los siguientes envíos:
Envíe desde la fábrica en CDMX a la sucursal en GDA : 943.0 toneladas
Envíe desde la fábrica en CDMX a la sucursal en AGU : 714.0 toneladas
Envíe desde la fábrica en CDMX a la sucursal en PVA : 1885.0 toneladas
Envíe desde la fábrica en CDMX a la sucursal en LEO : 1508.0 toneladas
Envíe desde la fábrica en MTY a la sucursal en ZAC : 1320.0 toneladas
Envíe desde la fábrica en MTY a la sucursal en AGU : 980.0 toneladas
Envíe desde la fábrica en MTY a la sucursal en TOR : 1700.0 toneladas


In [31]:
# Análisis de Sensibilidad por Fábrica
# Vamos a crear una tabla que contenga la Holgura y Precio Sombra para cada una de las fábricas

Holgura cero significa que se usó toda
la capacidad de las plantas.

El prcio sombra de -156 dice que por cada tonelada adicional que tuvieramos en CDMX se podrian ahorrar -156 pesos.

Si se quisiera aumentar la capacidad de alguna de las plantas la mejor opción sería CDMX

In [32]:
print("Análisis de Sensibilidad por Fábrica")
print("Fábrica          Holgura Precio Sombra")
for i in modelo.restriccionfabrica.keys():
    f = FABRICAS[i-1]
    print("{0:10s}{1:10.1f}{2:10.1f}".format(f, (Oferta[f] - modelo.restriccionfabrica[i]()), modelo.dual[modelo.restriccionfabrica[i]]))

Análisis de Sensibilidad por Fábrica
Fábrica          Holgura Precio Sombra
CDMX             0.0    -156.0
MTY              0.0       0.0


In [33]:
FABRICAS[1]

'MTY'

In [34]:
modelo.restriccionfabrica[2]()

4000.0

In [35]:
# Análisis de Sensibilidad por Sucursal

Las cantidades enviadas a las sucursales son iguales a las cantidades demandas

El precio sombra nos dice cuanto sería el costo por alguna unidad adicional de demanda



In [36]:
print("Análisis de Sensibilidad por Sucursal")
print("Sucursal       Holgura  Precio Sombra")
for j in modelo.restriccionsucursal.keys():
    s = SUCURSALES[j-1]
    print("{0:10s}{1:10.1f}{2:10.1f}".format(s, (modelo.restriccionsucursal[j]() - Demanda[s]), modelo.dual[modelo.restriccionsucursal[j]]))

Análisis de Sensibilidad por Sucursal
Sucursal       Holgura  Precio Sombra
GDA              0.0    1258.0
ZAC              0.0     930.0
AGU              0.0    1146.0
PVA              0.0    1918.0
TOR              0.0     668.0
LEO              0.0     914.0
